In [ ]:
import couchdb
import pandas as pd 
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns

from dotenv import load_dotenv
from IPython.display import display

## 1. Verbindung zur existierenden Datenbank

In [ ]:
load_dotenv(dotenv_path='.env')
COUCHDB_USER = os.getenv("COUCHDB_USER")
COUCHDB_PASSWORD = os.getenv("COUCHDB_PASSWORD")
COUCHDB_HOST = "localhost:5984"  
COUCHDB_URL = f"http://{COUCHDB_USER}:{COUCHDB_PASSWORD}@{COUCHDB_HOST}"
DB_NAME = 'world_factbook' 

try: 
    server = couchdb.Server(COUCHDB_URL)
    print(f"Erfolgreich verbunden mit CouchDB unter {COUCHDB_URL}")
    
    if DB_NAME in server:
        db = server[DB_NAME]
        print(f"Datenbank {DB_NAME} erfolgreich ausgewählt")
    else:
        print(f"Datenbank {DB_NAME} nicht gefunden!")
        raise LookupError(f"Datenbank {DB_NAME} nicht gefunden!")

except Exception as e:
    print(f"Fehler beim Verbinden mit CouchDB: {e}")
    raise e

### Hilfsfunktion zum Erstellen/Aktualisieren von Design-Dokumenten

In [ ]:
def sync_design_doc(db_handle, design_doc_dict):
    if not db_handle:
        print("sync_design_doc: Keine Datenbankverbindung.")
        return False
        
    doc_id = design_doc_dict['_id']
    try:
        existing_doc = db_handle.get(doc_id)
        if existing_doc:
            if existing_doc.get('views') != design_doc_dict.get('views') or \
               existing_doc.get('language') != design_doc_dict.get('language'):
                design_doc_dict['_rev'] = existing_doc['_rev'] 
                db_handle.save(design_doc_dict)
                print(f"Design-Dokument '{doc_id}' aktualisiert.")
                return True
            else:
                print(f"Design-Dokument '{doc_id}' ist bereits aktuell.")
                return False
        else:
            # Design-Dokument existiert nicht, neu erstellen
            db_handle.save(design_doc_dict)
            print(f"Design-Dokument '{doc_id}' erstellt.")
            return True
    except Exception as e:
        print(f"Fehler beim Synchronisieren des Design-Dokuments '{doc_id}': {e}")
        return False

## Datenauswertung

## 1. Analyse: Wirtschaftliche Dynamik und Ungleichheit 

**Fragestellung:** Welche Länder verzeichnen ein hohes reales Wirtschaftswachstum, weisen aber gleichzeitig eine hohe Einkommensungleichheit (gemessen am Gini-Koeffizienten) auf?

**Ziel der Analyse:** Diese Analyse soll Länder identifizieren, bei denen der generierte Wohlstand möglicherweise nicht breit in der Bevölkerung ankommt oder bei denen schnelles Wirtschaftswachstum mit einer Zunahme der Ungleichheit einhergeht. Dies kann Anstöße für Diskussionen über Verteilungsgerechtigkeit und die Nachhaltigkeit von Wachstumsmodellen geben.

### 1.1 **CouchDB-Technik: Map View**
Erstellung einer Map View namens `map_function_geografische_region_agg`.

**Map-Funktion:** 
- Eine Map-Funktion filtert Länder anhand festgelegter GDP/Gini-Schwellenwerte und emittiert Kontinent als Key und {Land, gdp, gini} als Value.
- Eine Reduce-Funktion sammelt die Länderdetails pro Kontinent.
- Python fragt die View mit group=True ab und erhält direkt die aggregierten Listen.


In [ ]:


wachstum_schwellenwert_for_view = 4.0
gini_schwellenwert_for_view = 40.0

if db:
    try:
        design_doc = db.get('_design/economic_analysis')
        if design_doc is None:
            design_doc = {'_id': '_design/economic_analysis', 'language': 'javascript', 'views': {}}
    except Exception:
        design_doc = {'_id': '_design/economic_analysis', 'language': 'javascript', 'views': {}}

    # Map-Funktion zur Filterung
    map_function_geografische_region_agg = f"""
    function(doc) {{
        var WACHSTUM_SCHWELLE = {wachstum_schwellenwert_for_view};
        var GINI_SCHWELLE = {gini_schwellenwert_for_view};
        if (doc.type === 'country' &&
            doc["Economy: Real GDP growth rate"] &&
            doc["Economy: Gini Index coefficient - distribution of family income"] &&
            doc["Government: Country name - conventional short form"] &&
            doc["Geography: Map references"]) {{
            var gdp_growth_str = doc["Economy: Real GDP growth rate"];
            var gini_str = doc["Economy: Gini Index coefficient - distribution of family income"];
            var country_name = doc["Government: Country name - conventional short form"];
            var geografische_region = doc["Geography: Map references"];
            var parsed_gdp_growth = null;
            var parsed_gini = null;
            if (typeof gdp_growth_str === 'string') {{
                var match_growth = gdp_growth_str.match(/(-?\\d+\\.?\\d*|-?\\d*\\.\\d+)/);
                if (match_growth && match_growth[0]) {{
                    parsed_gdp_growth = parseFloat(match_growth[0]);
                }}
            }} else if (typeof gdp_growth_str === 'number') 
            {{ parsed_gdp_growth = gdp_growth_str; }}
            if (typeof gini_str === 'string') {{
                var match_gini = gini_str.match(/(\\d+\\.?\\d*|\\d*\\.\\d+)/);
                if (match_gini && match_gini[0]) {{
                    parsed_gini = parseFloat(match_gini[0]);
                }}
            }} else if (typeof gini_str === 'number') 
            {{ parsed_gini = gini_str; }}
            
            // Filterung nach Kriterien
            if (parsed_gdp_growth !== null && !isNaN(parsed_gdp_growth) &&
                parsed_gini !== null && !isNaN(parsed_gini) &&
                country_name && geografische_region) {{
                if (parsed_gdp_growth > WACHSTUM_SCHWELLE && parsed_gini > GINI_SCHWELLE) {{
                    emit(geografische_region, {{ "country": country_name, "gdp_growth": parsed_gdp_growth, "gini": parsed_gini }});
                }}
            }}
        }}
    }}
    """

    #* Reduce-Funktion zur Aggregation pro geografische Region
    reduce_function_geografische_region_agg = f"""
    function(keys, values, rereduce) {{
        var all_country_data = [];
        if (rereduce) {{
            for (var i = 0; i < values.length; i++) {{
                all_country_data = all_country_data.concat(values[i]);
            }}
        }} else {{
            all_country_data = values;
        }}
        return all_country_data;
    }}
    """
    
    # --- Die neue View ---
    if 'views' not in design_doc:
        design_doc['views'] = {}

    design_doc['views']['countries_by_geografische_region_filtered'] = {
        'map': map_function_geografische_region_agg,
        'reduce': reduce_function_geografische_region_agg
    }
    
    # Design-Dokument in CouchDB aktualisieren
    sync_design_doc(db, design_doc)
else:
    print("Keine Datenbankverbindung ('db'), Design-Dokument konnte nicht aktualisiert werden.")

In [ ]:
wachstum_schwellenwert_for_view = 4.0
gini_schwellenwert_for_view = 40.0

print(f"--- Aggregation nach geografischer Region via 'countries_by_geografische_region_filtered' ---")
print(f"Filterkriterien in der View: Wachstum > {wachstum_schwellenwert_for_view}% UND Gini Index > {gini_schwellenwert_for_view}")

aggregated_results = []
try:
    view_results = db.view('economic_analysis/countries_by_geografische_region_filtered', group=True)
    for row in view_results:
        aggregated_results.append(row)
except Exception as e:
    print(f"Fehler beim Abfragen der View 'economic_analysis/countries_by_geografische_region_filtered': {e}")

# Initialisiere die DataFrame-Variable mit None oder einem leeren DataFrame
df_geografische_region_summary_plot = pd.DataFrame()

if aggregated_results:
    print(f"\n{len(aggregated_results)} geografische Regionen haben Länder, die die Kriterien erfüllen.")

    all_country_details = []
    geografische_region_summary_list = []

    for row in sorted(aggregated_results, key=lambda x: x.key):
        geografische_region = row.key
        countries_list = row.value

        if countries_list and isinstance(countries_list, list):
            countries_list_sorted = sorted(countries_list, key=lambda x: x.get('country', ''))
            
            for country_info in countries_list_sorted:
                all_country_details.append({
                    'Geografische Region': geografische_region,
                    'Land': country_info.get('country'),
                    'Wachstumsrate (%)': country_info.get('gdp_growth'),
                    'Gini-Index': country_info.get('gini')
                })
            
            valid_gdp = [c['gdp_growth'] for c in countries_list if isinstance(c.get('gdp_growth'), (int, float))]
            valid_gini = [c['gini'] for c in countries_list if isinstance(c.get('gini'), (int, float))]
            
            avg_growth = round(np.mean(valid_gdp), 2) if valid_gdp else np.nan
            avg_gini = round(np.mean(valid_gini), 2) if valid_gini else np.nan

            geografische_region_summary_list.append({
                'Geografische Region': geografische_region,
                'Anzahl Länder im Profil': len(countries_list),
                'Länder': ', '.join([c.get('country', '') for c in countries_list_sorted]),
                'Durchschn. Wachstum (%)': avg_growth,
                'Durchschn. Gini-Index': avg_gini
            })

    if all_country_details:
        df_display_all_countries = pd.DataFrame(all_country_details)
        print("\n--- Gesamttabelle der gefilterten Länder ---")
        display(df_display_all_countries)
        
else:
    print("\nKeine Länder im Profil gefunden. Es wird keine Tabelle oder Grafik für Analyse 1 erstellt.")

### 1.2 **Abfrage der View aus Python:**

### 1.3 Aggregation der Ergebnisse

In [ ]:
if geografische_region_summary_list:
    df_geografische_region_summary = pd.DataFrame(geografische_region_summary_list)
    df_geografische_region_summary_plot = df_geografische_region_summary[(
        df_geografische_region_summary['Durchschn. Wachstum (%)'] != 'N/A') &
        (df_geografische_region_summary['Durchschn. Gini-Index'] != 'N/A')
    ].copy()  # .copy() um SettingWithCopyWarning zu vermeiden

    # Konvertiere Spalten zu numerisch für den Plot, falls sie gemischte Typen haben könnten
    df_geografische_region_summary_plot['Anzahl Länder im Profil'] = pd.to_numeric(df_geografische_region_summary_plot['Anzahl Länder im Profil'])

    df_geografische_region_summary_plot = df_geografische_region_summary_plot.sort_values(by='Anzahl Länder im Profil', ascending=False)

    print("\n--- Zusammenfassung nach geografischer Region (aus CouchDB Aggregation) ---")
    display(df_geografische_region_summary[['Geografische Region', 'Anzahl Länder im Profil', 'Durchschn. Wachstum (%)', 'Durchschn. Gini-Index', 'Länder']])


### 1.4 Visualisierung der Ergebnisse

In [ ]:
# Visualisierung
plt.figure(figsize=(10, 6))
sns.barplot(data=df_geografische_region_summary_plot,
            x='Anzahl Länder im Profil', y='Geografische Region', palette='viridis', hue='Geografische Region', dodge=False)
plt.title(f'Anzahl Länder pro geografischer Region (Wachstum > {wachstum_schwellenwert_for_view}% & Gini > {gini_schwellenwert_for_view})', fontsize=14)
plt.xlabel('Anzahl Länder im Profil', fontsize=12)
plt.ylabel('Geografische Region', fontsize=12)
plt.tight_layout()
plt.show()

## 2. Analyse: Bildungsrendite-Paradoxon

Diese Analyse untersucht das Phänomen des "Bildungsrendite-Paradoxons". Es sollen Länder identifiziert werden, die signifikante Bildungsinvestitionen tätigen (approximiert durch Ausgaben und Ausbildungsdauer), aber gleichzeitig Indikatoren für potenziellen "Brain-Drain" oder mangelnde inlandige Perspektiven aufweisen (approximiert durch Netto-Migrationsrate und Jugendarbeitslosigkeit). 
Die Kernfrage ist: **Welche Länder zeigen ein Profil hoher Bildungswerte bei gleichzeitig Anzeichen für hohen Abwanderungsdruck?** Die Analyse nutzt serverseitiges Parsen der Rohdaten in CouchDB und eine flexible, clientseitige Kategorisierung und Filterung der Profile in Python.

**Datenbasis und relevante Felder:**
Die Analyse basiert auf folgenden Feldern aus dem CIA World Factbook Datensatz, die serverseitig aus ihren ursprünglichen String-Formaten in numerische Werte umgewandelt werden:
* `People and Society: Education expenditures` (als % des BIP)
* `People and Society: School life expectancy (primary to tertiary education) - total` (in Jahren)
* `People and Society: Net migration rate` (pro 1.000 Einwohner)
* `Economy: Youth unemployment rate (ages 15-24) - total` (als %)
* `Government: Country name - conventional short form` (zur Identifikation)
* `type`: Ein beim Import hinzugefügtes Feld zur Filterung auf Dokumente vom Typ 'country'.


### 2.1 **Methodisches Vorgehen:**

Serverseitige Datenaufbereitung (Parsing) mit CouchDB MapReduce View

Um eine saubere und numerische Datengrundlage für die Analyse in Python zu schaffen, wird eine CouchDB MapReduce View verwendet.
* **View-Name:** `parsed_education_migration_indicators` (im Design-Dokument `_design/indicator_parser`).
* **Map-Funktion (JavaScript):**
    1.  **Filterung:** Verarbeitet ausschließlich Dokumente vom `type: 'country'`.
    2.  **Existenzprüfung:** Stellt sicher, dass alle vier für die Analyse benötigten Rohfelder sowie der Ländername im Dokument vorhanden sind.
    3.  **Robustes Parsen:** Extrahiert für jeden der vier Indikatoren den ersten gefundenen numerischen Wert aus dem jeweiligen String-Feld (z.B. aus "2.9% of GDP (2020 est.)" wird die Zahl `2.9`). Die Funktion `parseNum` innerhalb der Map-Funktion nutzt dazu reguläre Ausdrücke (`String.match()`) und `parseFloat()`. "NA"-Werte oder nicht erfolgreich parsierbare Strings resultieren in `null`.
    4.  **Emission:** Wenn alle vier Indikatoren erfolgreich zu Zahlen geparst werden konnten, emittiert die Funktion:



In [ ]:
if db: 
    map_function_js_parser = f"""
    function(doc) {{
        if (doc.type === 'country' &&
            doc["Government: Country name - conventional short form"] &&
            doc["People and Society: Education expenditures"] &&
            doc["People and Society: School life expectancy (primary to tertiary education) - total"] &&
            doc["People and Society: Net migration rate"] &&
            doc["Economy: Youth unemployment rate (ages 15-24) - total"]) {{

            var country_name = doc["Government: Country name - conventional short form"];
            var edu_exp_str = doc["People and Society: Education expenditures"];
            var school_exp_str = doc["People and Society: School life expectancy (primary to tertiary education) - total"];
            var migration_str = doc["People and Society: Net migration rate"];
            var youth_unempl_str = doc["Economy: Youth unemployment rate (ages 15-24) - total"];

            function parseNum(strVal) {{
                if (strVal === "NA" || strVal === null || typeof strVal === 'undefined') return null;
                if (typeof strVal === 'number') return strVal;
                if (typeof strVal === 'string') {{
                    var match = strVal.match(/(-?\\d+\\.?\\d*|-?\\d*\\.?\\d+)/);
                    if (match && match[0]) return parseFloat(match[0]);
                }}
                return null; 
            }}

            var num_edu_expenditures = parseNum(edu_exp_str);
            var num_school_expectancy = parseNum(school_exp_str);
            var num_migration = parseNum(migration_str);
            var num_youth_unemployment = parseNum(youth_unempl_str);

            if (num_edu_expenditures !== null && num_school_expectancy !== null && num_migration !== null && num_youth_unemployment !== null) {{
                emit(country_name, {{ 
                    Bildungsausgaben: num_edu_expenditures,
                    Schullebensdauer: num_school_expectancy,
                    Migration: num_migration,
                    Jugendarbeitslosigkeit: num_youth_unemployment
                }});
            }}
        }}
    }}
    """
    design_doc_parser = {
        '_id': '_design/indicator_parser', 
        'language': 'javascript',
        'views': {
            'parsed_education_migration_indicators': {
                'map': map_function_js_parser
            }
        }
    }
    sync_design_doc(db, design_doc_parser)
else:
    print("Keine Datenbankverbindung.")

### 2.2 **Clientseitige Kategorisierung, Filterung und Visualisierung (Python)**

**Definition von Schwellenwerten für Kategorien:**
    
- Für jeden der vier numerischen Indikatoren werden Schwellenwerte definiert, um die Länder in diskrete Kategorien ("Niedrig", "Mittel", "Hoch" bzw. spezifischere Bezeichnungen für Migration) einzuteilen.

**Filterung des "Paradoxon"-Profils:**
- Es wird ein spezifisches "Paradoxon"-Profil definiert, z.B. Länder mit hohen Bildungsausgaben, langer Schullebenserwartung, stark negativer Migrationsrate und hoher Jugendarbeitslosigkeit.
- Der Pandas DataFrame `df_parsed` wird nach dieser exakten Kombination von Kategorien gefiltert.


In [ ]:
# Daten aus der View abrufen 
print("\nGeparste Indikatordaten CouchDB-View...")
all_countries_parsed_data = []
try:
    results = db.view(
        'indicator_parser/parsed_education_migration_indicators',
    )
    for row in results:
        data = row.value
        data['country'] = row.key 
        all_countries_parsed_data.append(data)
    
    if not all_countries_parsed_data:
        print("Keine Daten aus der View 'parsed_education_migration_indicators' erhalten.")
    else:
        print(f"{len(all_countries_parsed_data)} Länderdatensätze aus View geladen.")
except Exception as e:
    print(f"Fehler beim Abrufen der View-Daten: {e}")
    all_countries_parsed_data = []

# Definition von Schwellenwerten 
if all_countries_parsed_data:
    df_parsed = pd.DataFrame(all_countries_parsed_data)
    
    # Bildungsausgaben (% des BIP)
    SCHWELLE_BILDUNG_NIEDRIG = 4.0
    SCHWELLE_BILDUNG_HOCH = 6.0
    
    # Schullebenserwartung (Jahre)
    SCHWELLE_SCHULLEBEN_KURZ = 8.0
    SCHWELLE_SCHULLEBEN_LANG = 15.0
    
    # Netto-Migrationsrate (pro 1.000 Einwohner)
    SCHWELLE_MIGRATION_NEGATIV = -1.0
    SCHWELLE_MIGRATION_AUSGEGLICHEN_MAX = 0.0    

    # Jugendarbeitslosigkeit (%)
    SCHWELLE_JUGENDARBEITSLOSIGKEIT_NIEDRIG = 7.0
    SCHWELLE_JUGENDARBEITSLOSIGKEIT_HOCH = 13.0

    print("\nSchwellenwerte für Kategorisierung:\n")
    print(f"  Bildungsausgaben (% BIP): \n\tNiedrig < {SCHWELLE_BILDUNG_NIEDRIG}, Mittel {SCHWELLE_BILDUNG_NIEDRIG}-{SCHWELLE_BILDUNG_HOCH}, Hoch >= {SCHWELLE_BILDUNG_HOCH}")
    print(f"  Schullebenserwartung (Jahre): \n\tKurz < {SCHWELLE_SCHULLEBEN_KURZ}, Mittel {SCHWELLE_SCHULLEBEN_KURZ}-{SCHWELLE_SCHULLEBEN_LANG}, Lang >= {SCHWELLE_SCHULLEBEN_LANG}")
    print(f"  Netto-Migrationsrate (pro 1000): \n\tNegativ < {SCHWELLE_MIGRATION_NEGATIV}, Ausgeglichen {SCHWELLE_MIGRATION_NEGATIV} bis {SCHWELLE_MIGRATION_AUSGEGLICHEN_MAX}, Positiv > {SCHWELLE_MIGRATION_AUSGEGLICHEN_MAX}")
    print(f"  Jugendarbeitslosigkeit (%): \n\tNiedrig < {SCHWELLE_JUGENDARBEITSLOSIGKEIT_NIEDRIG}, Mittel {SCHWELLE_JUGENDARBEITSLOSIGKEIT_NIEDRIG}-{SCHWELLE_JUGENDARBEITSLOSIGKEIT_HOCH}, Hoch >= {SCHWELLE_JUGENDARBEITSLOSIGKEIT_HOCH}")

    # Kategorisierung
    def categorize_value(value, min_value, max_value, category_low="Niedrig", category_mid="Mittel", category_high="Hoch"):
        if pd.isna(value): return "Unbekannt"
        if value < min_value: return category_low
        if value >= max_value: return category_high
        return category_mid

    df_parsed['bildungsausgaben_kat'] = df_parsed['Bildungsausgaben'].apply(lambda x: categorize_value(x, SCHWELLE_BILDUNG_NIEDRIG, SCHWELLE_BILDUNG_HOCH))
    df_parsed['schullebenserwartung_kat'] = df_parsed['Schullebensdauer'].apply(lambda x: categorize_value(x, SCHWELLE_SCHULLEBEN_KURZ, SCHWELLE_SCHULLEBEN_LANG, category_low="Kurz", category_high="Lang"))

    def categorize_migration(value, strong_negative, neutral_balance_limit, category_low="Negativ", category_mid="Mittel", optimal_balance="Ausgeglichen"):
        if pd.isna(value): return "Unbekannt"
        if value < strong_negative: return category_low
        if value <= neutral_balance_limit: return category_mid
        return optimal_balance

    df_parsed['migrationsrate_kat'] = df_parsed['Migration'].apply(lambda x: categorize_migration(x, SCHWELLE_MIGRATION_NEGATIV, SCHWELLE_MIGRATION_AUSGEGLICHEN_MAX))
    df_parsed['jugendarbeitslosigkeit_kat'] = df_parsed['Jugendarbeitslosigkeit'].apply(lambda x: categorize_value(x, SCHWELLE_JUGENDARBEITSLOSIGKEIT_NIEDRIG, SCHWELLE_JUGENDARBEITSLOSIGKEIT_HOCH, category_low="Niedrig", category_mid="Mittel", category_high="Hoch"))

    # Filterung des "Paradoxon"-Profils
    edu_expenditure_profile = "Hoch"
    school_life_expectancy_profile = "Lang"  
    migration_profile = "Negativ"
    youth_unemployment_profile = "Hoch"

    print(f"\nLänder mit dem primären Zielprofil: \n\tBildungsausgaben='{edu_expenditure_profile}', \n\tSchullebenserwartung='{school_life_expectancy_profile}', \n\tMigration='{migration_profile}', \n\tJugendarbeitslosigkeit='{youth_unemployment_profile}'")

    df_paradox_countries = df_parsed[
        (df_parsed['bildungsausgaben_kat'] == edu_expenditure_profile) &
        (df_parsed['schullebenserwartung_kat'] == school_life_expectancy_profile) &
        (df_parsed['migrationsrate_kat'] == migration_profile) &
        (df_parsed['jugendarbeitslosigkeit_kat'] == youth_unemployment_profile)
    ]
    
    data_for_plot = None 

    if not df_paradox_countries.empty:
        print(f"\n{len(df_paradox_countries)} Länder im primären 'Paradoxon'-Profil gefunden:")
        display(df_paradox_countries[['country', 'Bildungsausgaben', 'Schullebensdauer', 'Migration', 'Jugendarbeitslosigkeit']].sort_values(by=['Migration', 'Jugendarbeitslosigkeit'], ascending=[True, False]))
        data_for_plot = df_paradox_countries      
    else:
        print("\nKeine Länder im primären 'Paradoxon'-Profil gefunden...")

### 2.3 **Grafische Darstellung:**

   -  Die Länder, die einem der identifizierten Profile entsprechen, werden in einem Streudiagramm (Scatter Plot) visualisiert.
   -  **Achsen:** Jugendarbeitslosigkeit vs. Netto-Migrationsrate.
   -  Elemente der Grafik sind ein  Titel, klare Achsenbeschriftungen, eine Legende und die Annotation einiger Länderpunkte zur besseren Lesbarkeit.

In [ ]:
if data_for_plot is not None:
    if len(data_for_plot) > 0 : 
        plt.figure(figsize=(15, 10))
        sns.scatterplot(
            data=data_for_plot,
            x='Jugendarbeitslosigkeit',
            y='Migration',
            size='Bildungsausgaben', 
            hue='Schullebensdauer',
            sizes=(100, 700),
            alpha=0.7,
            legend='full'
        )
        
        plt.title(f"Länderprofile zum Bildungsrendite-Paradoxon", fontsize=14, fontweight='bold')
        plt.xlabel('Jugendarbeitslosigkeit (%)', fontsize=12)
        plt.ylabel('Netto-Migrationsrate (pro 1000 Einwohner)', fontsize=12)
        plt.axhline(0, color='black', linestyle='--', linewidth=0.8, label='Migrationsrate = 0')
        
        # Linien für die aktuellen Schwellenwerte des gefilterten Profils
        plt.axvline(SCHWELLE_JUGENDARBEITSLOSIGKEIT_HOCH if youth_unemployment_profile == "Hoch" 
                    else (SCHWELLE_JUGENDARBEITSLOSIGKEIT_NIEDRIG if youth_unemployment_profile == "Mittel" 
                            else -np.inf) , color='red', linestyle=':', linewidth=0.7, label=f"Schwelle Jugendarbeitslosigkeit für Kategorie '{youth_unemployment_profile}'")
        plt.axhline(SCHWELLE_MIGRATION_NEGATIV if migration_profile == "Negativ" 
                    else (SCHWELLE_MIGRATION_AUSGEGLICHEN_MAX if migration_profile == "Ausgeglichen" 
                            else np.inf), color='blue', linestyle=':', linewidth=0.7, label=f"Schwelle Migration für Kategorie '{migration_profile}'")

        num_to_annotate = min(len(data_for_plot), 7)
        if num_to_annotate > 0:
            for i in range(num_to_annotate):
                row = data_for_plot.iloc[i]
                plt.text(row['Jugendarbeitslosigkeit'] + 0.3, 
                            row['Migration'] + 0.3, 
                            row['country'], fontdict={'size': 9})

        plt.legend(title='Legende', bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.,  labelspacing=2.0)
        plt.grid(True, linestyle=':', alpha=0.6)
        plt.tight_layout(rect=[0, 0, 0.80, 1]) 
        plt.show()
    else:
        print("Nicht genügend Datenpunkte (<=1) im ausgewählten Profil für einen sinnvollen Scatterplot.")

In [ ]:
if db: 
    # Schwellenwerte für die Kategorisierung
    SCHWELLE_BILDUNG_NIEDRIG = 4.0
    SCHWELLE_BILDUNG_HOCH = 6.0
    SCHWELLE_SCHULLEBEN_KURZ = 8.0
    SCHWELLE_SCHULLEBEN_LANG = 15.0
    SCHWELLE_MIGRATION_NEGATIV = -1.0
    SCHWELLE_MIGRATION_AUSGEGLICHEN_MAX = 0.0
    SCHWELLE_JUGENDARBEITSLOSIGKEIT_NIEDRIG = 7.0
    SCHWELLE_JUGENDARBEITSLOSIGKEIT_HOCH = 13.0
    
    # Definition des Paradoxon-Profils
    PARADOXON_BILDUNG = "Hoch"
    PARADOXON_SCHULE = "Lang"
    PARADOXON_MIGRATION = "Negativ"
    PARADOXON_JUGENDARB = "Hoch"
    
    map_function_js_categorized = f"""
    function(doc) {{
        if (doc.type === 'country' &&
            doc["Government: Country name - conventional short form"] &&
            doc["People and Society: Education expenditures"] &&
            doc["People and Society: School life expectancy (primary to tertiary education) - total"] &&
            doc["People and Society: Net migration rate"] &&
            doc["Economy: Youth unemployment rate (ages 15-24) - total"]) {{

            var country_name = doc["Government: Country name - conventional short form"];
            var edu_exp_str = doc["People and Society: Education expenditures"];
            var school_exp_str = doc["People and Society: School life expectancy (primary to tertiary education) - total"];
            var migration_str = doc["People and Society: Net migration rate"];
            var youth_unempl_str = doc["Economy: Youth unemployment rate (ages 15-24) - total"];
            var geografische_region = doc["Geography: Map references"] || "Unknown";

            function parseNum(strVal) {{
                if (strVal === "NA" || strVal === null || typeof strVal === 'undefined') return null;
                if (typeof strVal === 'number') return strVal;
                if (typeof strVal === 'string') {{
                    var match = strVal.match(/(-?\\d+\\.?\\d*|-?\\d*\\.?\\d+)/);
                    if (match && match[0]) return parseFloat(match[0]);
                }}
                return null; 
            }}

            var num_edu_expenditures = parseNum(edu_exp_str);
            var num_school_expectancy = parseNum(school_exp_str);
            var num_migration = parseNum(migration_str);
            var num_youth_unemployment = parseNum(youth_unempl_str);

            if (num_edu_expenditures !== null && num_school_expectancy !== null && 
                num_migration !== null && num_youth_unemployment !== null) {{
                
                // Kategorisierungsfunktionen
                function categorize_value(value, min_value, max_value, category_low, category_mid, category_high) {{
                    if (value === null || isNaN(value)) return "Unbekannt";
                    if (value < min_value) return category_low;
                    if (value >= max_value) return category_high;
                    return category_mid;
                }}
                
                function categorize_migration(value, strong_negative, neutral_balance_limit) {{
                    if (value === null || isNaN(value)) return "Unbekannt";
                    if (value < strong_negative) return "Negativ";
                    if (value <= neutral_balance_limit) return "Mittel";
                    return "Ausgeglichen";
                }}
                
                // Kategorien berechnen
                var bildungsausgaben_kat = categorize_value(
                    num_edu_expenditures, 
                    {SCHWELLE_BILDUNG_NIEDRIG}, 
                    {SCHWELLE_BILDUNG_HOCH}, 
                    "Niedrig", "Mittel", "Hoch"
                );
                
                var schullebenserwartung_kat = categorize_value(
                    num_school_expectancy, 
                    {SCHWELLE_SCHULLEBEN_KURZ}, 
                    {SCHWELLE_SCHULLEBEN_LANG}, 
                    "Kurz", "Mittel", "Lang"
                );
                
                var migrationsrate_kat = categorize_migration(
                    num_migration, 
                    {SCHWELLE_MIGRATION_NEGATIV}, 
                    {SCHWELLE_MIGRATION_AUSGEGLICHEN_MAX}
                );
                
                var jugendarbeitslosigkeit_kat = categorize_value(
                    num_youth_unemployment, 
                    {SCHWELLE_JUGENDARBEITSLOSIGKEIT_NIEDRIG}, 
                    {SCHWELLE_JUGENDARBEITSLOSIGKEIT_HOCH}, 
                    "Niedrig", "Mittel", "Hoch"
                );
                
                var matches_paradox_profile = (
                    bildungsausgaben_kat === "{PARADOXON_BILDUNG}" && 
                    schullebenserwartung_kat === "{PARADOXON_SCHULE}" && 
                    migrationsrate_kat === "{PARADOXON_MIGRATION}" && 
                    jugendarbeitslosigkeit_kat === "{PARADOXON_JUGENDARB}"
                );
                
                // Emittiere für alle Länder mit kompletten Daten
                emit(country_name, {{
                    geografische_region: geografische_region,
                    Bildungsausgaben: num_edu_expenditures,
                    Schullebensdauer: num_school_expectancy,
                    Migration: num_migration,
                    Jugendarbeitslosigkeit: num_youth_unemployment,
                    bildungsausgaben_kat: bildungsausgaben_kat,
                    schullebenserwartung_kat: schullebenserwartung_kat,
                    migrationsrate_kat: migrationsrate_kat,
                    jugendarbeitslosigkeit_kat: jugendarbeitslosigkeit_kat,
                    is_paradox_profile: matches_paradox_profile
                }});
            }}
        }}
    }}
    """
    
    # Zusätzliche View für direkte Abfrage des Paradoxon-Profils
    map_function_js_paradox = f"""
    function(doc) {{
        if (doc.type === 'country' &&
            doc["Government: Country name - conventional short form"] &&
            doc["People and Society: Education expenditures"] &&
            doc["People and Society: School life expectancy (primary to tertiary education) - total"] &&
            doc["People and Society: Net migration rate"] &&
            doc["Economy: Youth unemployment rate (ages 15-24) - total"]) {{

            var country_name = doc["Government: Country name - conventional short form"];
            var edu_exp_str = doc["People and Society: Education expenditures"];
            var school_exp_str = doc["People and Society: School life expectancy (primary to tertiary education) - total"];
            var migration_str = doc["People and Society: Net migration rate"];
            var youth_unempl_str = doc["Economy: Youth unemployment rate (ages 15-24) - total"];
            var geografische_region = doc["Geography: Map references"] || "Unknown";

            function parseNum(strVal) {{
                if (strVal === "NA" || strVal === null || typeof strVal === 'undefined') return null;
                if (typeof strVal === 'number') return strVal;
                if (typeof strVal === 'string') {{
                    var match = strVal.match(/(-?\\d+\\.?\\d*|-?\\d*\\.?\\d+)/);
                    if (match && match[0]) return parseFloat(match[0]);
                }}
                return null; 
            }}

            var num_edu_expenditures = parseNum(edu_exp_str);
            var num_school_expectancy = parseNum(school_exp_str);
            var num_migration = parseNum(migration_str);
            var num_youth_unemployment = parseNum(youth_unempl_str);

            if (num_edu_expenditures !== null && num_school_expectancy !== null && 
                num_migration !== null && num_youth_unemployment !== null) {{
                
                // Direkte Schwellenwertprüfung für das Paradoxon-Profil
                if (num_edu_expenditures >= {SCHWELLE_BILDUNG_HOCH} && 
                    num_school_expectancy >= {SCHWELLE_SCHULLEBEN_LANG} &&
                    num_migration < {SCHWELLE_MIGRATION_NEGATIV} &&
                    num_youth_unemployment >= {SCHWELLE_JUGENDARBEITSLOSIGKEIT_HOCH}) {{
                    
                    emit(country_name, {{ 
                        Bildungsausgaben: num_edu_expenditures,
                        Schullebensdauer: num_school_expectancy,
                        Migration: num_migration,
                        Jugendarbeitslosigkeit: num_youth_unemployment,
                        geografische_region: geografische_region
                    }});
                }}
            }}
        }}
    }}
    """

    # Gruppierung nach geografischer Region für Paradoxon-Länder
    map_function_js_paradox_by_geografische_region = f"""
    function(doc) {{
        if (doc.type === 'country' &&
            doc["Government: Country name - conventional short form"] &&
            doc["People and Society: Education expenditures"] &&
            doc["People and Society: School life expectancy (primary to tertiary education) - total"] &&
            doc["People and Society: Net migration rate"] &&
            doc["Economy: Youth unemployment rate (ages 15-24) - total"] &&
            doc["Geography: Map references"]) {{

            var country_name = doc["Government: Country name - conventional short form"];
            var edu_exp_str = doc["People and Society: Education expenditures"];
            var school_exp_str = doc["People and Society: School life expectancy (primary to tertiary education) - total"];
            var migration_str = doc["People and Society: Net migration rate"];
            var youth_unempl_str = doc["Economy: Youth unemployment rate (ages 15-24) - total"];
            var geografische_region = doc["Geography: Map references"];

            function parseNum(strVal) {{
                if (strVal === "NA" || strVal === null || typeof strVal === 'undefined') return null;
                if (typeof strVal === 'number') return strVal;
                if (typeof strVal === 'string') {{
                    var match = strVal.match(/(-?\\d+\\.?\\d*|-?\\d*\\.?\\d+)/);
                    if (match && match[0]) return parseFloat(match[0]);
                }}
                return null; 
            }}

            var num_edu_expenditures = parseNum(edu_exp_str);
            var num_school_expectancy = parseNum(school_exp_str);
            var num_migration = parseNum(migration_str);
            var num_youth_unemployment = parseNum(youth_unempl_str);

            if (num_edu_expenditures !== null && num_school_expectancy !== null && 
                num_migration !== null && num_youth_unemployment !== null && geografische_region) {{
                
                // Direkte Schwellenwertprüfung für das Paradoxon-Profil
                if (num_edu_expenditures >= {SCHWELLE_BILDUNG_HOCH} && 
                    num_school_expectancy >= {SCHWELLE_SCHULLEBEN_LANG} &&
                    num_migration < {SCHWELLE_MIGRATION_NEGATIV} &&
                    num_youth_unemployment >= {SCHWELLE_JUGENDARBEITSLOSIGKEIT_HOCH}) {{
                    
                    emit(geografische_region, {{ 
                        country: country_name,
                        Bildungsausgaben: num_edu_expenditures,
                        Schullebensdauer: num_school_expectancy,
                        Migration: num_migration,
                        Jugendarbeitslosigkeit: num_youth_unemployment
                    }});
                }}
            }}
        }}
    }}
    """

    # Reduce-Funktion für die geografische Regionaggregation
    reduce_function_geografische_region = """
    function(keys, values, rereduce) {
        var all_country_data = [];
        if (rereduce) {
            for (var i = 0; i < values.length; i++) {
                all_country_data = all_country_data.concat(values[i]);
            }
        } else {
            all_country_data = values;
        }
        return all_country_data;
    }
    """
    
    # Design-Dokument erstellen oder aktualisieren
    design_doc_education_paradox = {
        '_id': '_design/education_paradox_analysis', 
        'language': 'javascript',
        'views': {
            'categorized_education_indicators': {
                'map': map_function_js_categorized
            },
            'paradox_profile_countries': {
                'map': map_function_js_paradox
            },
            'paradox_countries_by_geografische_region': {
                'map': map_function_js_paradox_by_geografische_region,
                'reduce': reduce_function_geografische_region
            }
        }
    }
    
    # Design-Dokument in CouchDB speichern
    sync_design_doc(db, design_doc_education_paradox)   
    print("Design-Dokument mit serverseitiger Kategorisierung erstellt/aktualisiert.")
else:
    print("Keine Datenbankverbindung.")

In [ ]:
print("\nAbfrage der serverseitig kategorisierten Bildungsrendite-Paradox-Daten...")
paradox_countries = []
try:
    results = db.view('education_paradox_analysis/paradox_profile_countries')
    for row in results:
        data = row.value
        data['country'] = row.key 
        paradox_countries.append(data)
    
    if not paradox_countries:
        print("Keine Länder im Paradoxon-Profil gefunden.")
    else:
        print(f"{len(paradox_countries)} Länder im Paradoxon-Profil gefunden.")
        print("\n--- Schwellenwerte für Kategorisierung ---")
        print(f"  Bildungsausgaben (% BIP): \n\tNiedrig < {SCHWELLE_BILDUNG_NIEDRIG}, Mittel {SCHWELLE_BILDUNG_NIEDRIG}-{SCHWELLE_BILDUNG_HOCH}, Hoch >= {SCHWELLE_BILDUNG_HOCH}")
        print(f"  Schullebenserwartung (Jahre): \n\tKurz < {SCHWELLE_SCHULLEBEN_KURZ}, Mittel {SCHWELLE_SCHULLEBEN_KURZ}-{SCHWELLE_SCHULLEBEN_LANG}, Lang >= {SCHWELLE_SCHULLEBEN_LANG}")
        print(f"  Netto-Migrationsrate (pro 1000): \n\tNegativ < {SCHWELLE_MIGRATION_NEGATIV}, Ausgeglichen {SCHWELLE_MIGRATION_NEGATIV} bis {SCHWELLE_MIGRATION_AUSGEGLICHEN_MAX}, Positiv > {SCHWELLE_MIGRATION_AUSGEGLICHEN_MAX}")
        print(f"  Jugendarbeitslosigkeit (%): \n\tNiedrig < {SCHWELLE_JUGENDARBEITSLOSIGKEIT_NIEDRIG}, Mittel {SCHWELLE_JUGENDARBEITSLOSIGKEIT_NIEDRIG}-{SCHWELLE_JUGENDARBEITSLOSIGKEIT_HOCH}, Hoch >= {SCHWELLE_JUGENDARBEITSLOSIGKEIT_HOCH}")

        # Detailtabelle anzeigen
        df_server_paradox = pd.DataFrame(paradox_countries)
        print("\n--- Detailtabelle der Bildungsrendite-Paradoxon-Länder ---")
        df_display = df_server_paradox[['country', 'geografische_region', 'Bildungsausgaben', 'Schullebensdauer', 'Migration', 'Jugendarbeitslosigkeit']]
        df_display = df_display.sort_values(by=['Migration'], ascending=[True])
        display(df_display)

except Exception as e:
    print(f"Fehler beim Abrufen der Paradox-View-Daten: {e}")
    paradox_countries = []

In [ ]:
if paradox_countries:
    df_server_paradox = pd.DataFrame(paradox_countries)

plt.figure(figsize=(15, 10))
sns.scatterplot(
    data=df_server_paradox,
    x='Jugendarbeitslosigkeit',
    y='Migration',
    size='Bildungsausgaben', 
    hue='Schullebensdauer',
    sizes=(100, 700),
    alpha=0.7,
    legend='full'
)

plt.title(f"Länderprofile zum Bildungsrendite-Paradoxon", fontsize=14, fontweight='bold')
plt.xlabel('Jugendarbeitslosigkeit (%)', fontsize=12)
plt.ylabel('Netto-Migrationsrate (pro 1000 Einwohner)', fontsize=12)
plt.axhline(0, color='black', linestyle='--', linewidth=0.8, label='Migrationsrate = 0')

# Schwellenwertlinien einzeichnen
plt.axvline(SCHWELLE_JUGENDARBEITSLOSIGKEIT_HOCH, 
            color='red', linestyle=':', linewidth=0.7, 
            label=f"Schwelle Jugendarbeitslosigkeit: {SCHWELLE_JUGENDARBEITSLOSIGKEIT_HOCH}")
plt.axhline(SCHWELLE_MIGRATION_NEGATIV, 
            color='blue', linestyle=':', linewidth=0.7, 
            label=f"Schwelle Migration: {SCHWELLE_MIGRATION_NEGATIV}")

# Annotationen für Länderpunkte
num_to_annotate = min(len(df_server_paradox), 7)
if num_to_annotate > 0:
    for i in range(num_to_annotate):
        row = df_server_paradox.iloc[i]
        plt.text(
            row['Jugendarbeitslosigkeit'] + 0.1,
            row['Migration'] + 0.1,
            row['country'],
            fontdict={'size': 12}
        )

plt.legend(title='Legende', bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0., labelspacing=2.0)
plt.grid(True, linestyle=':', alpha=0.6)
plt.tight_layout(rect=[0, 0, 0.80, 1]) 
plt.show()